In [ ]:
# Organize imports and initialize variables
'''
TODO LIST:
= add exchange names
= search how to do pprint without sort
= buy/sell stocks
= live monitor
= check monitor
'''

from copy import deepcopy
from datetime import datetime
from pprint import pprint
from time import sleep
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

stocksList = requests.get('https://financialmodelingprep.com/api/v3/company/stock/list').json()

stock_count = 1

stocks_symbols = []
stocks_info = {}
current_stock_price = {}
stock_closing_info = {}
stock_closing_dates = []
stock_closing_prices = []
invalid_stocks = {}

day1Readings = []
day2Readings = []
day3Readings = []
day4Readings = []
day5Readings = []
day6Readings = []
currentReadings = []

In [ ]:
# HELPER FUNCTION
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In [ ]:
# Creates list of stocks represented by their symbols
# Creates a dictionary of current prices for each stock
for stock in stocksList['symbolsList']:
    stocks_symbols.append(stock['symbol'])
    if ('name' not in stock.keys() or stock['name'] == ""):
        current_stock_price[stock['symbol']] = stock["price"]
    else:
        stock_name = stock['symbol'] + ": " + stock['name']
        current_stock_price[stock_name] = stock["price"]

In [ ]:
# Creates an empty dictionary containing each stock's symbol and its readings
for stock in stocks_symbols:
    stocks_info[stock] = {"stock_name": stock,
                          "reading dates": [],
              "day 1 reading": "",
              "day 2 reading": "",
              "day 3 reading": "",
              "day 4 reading": "",
              "day 5 reading": "",
              "day 6 reading": "",
              "current reading": "",
                         "list of readings": []}

In [ ]:
# pprint(stocksList)
# pprint(stocks_prices)

# for stock in stocks_symbols[:25]:
for stock in stocks_symbols:
    # for stock in tqdm(stocks_symbols):
    # print(stock)
    print(f"Iterating through Stock {stock_count} out of {len(current_stock_price.items())}!")
    #     print(" ", end='\r')
    #     sleep(0.1)
    stock_data = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{stock}').json()
    if 'historical' in stock_data:
        for data in stock_data['historical'][-7:]:
            datetime_obj = datetime.strptime(data['date'], '%Y-%m-%d').strftime('%m-%d-%Y')
            stocks_info[stock]["reading dates"].append(datetime_obj)
            stock_closing_info[datetime_obj] = data['close']
    else:
        stock_closing_info["Stock has no ""historical data""!"] = 'Closing Price: NaN'

    stock_closing_dates = list(stock_closing_info.keys())
    stock_closing_prices = list(stock_closing_info.values())
        
    # print(stock, list(stock_closing_info.items())[0:7])
    
    stocks_info[stock]["day 1 reading"] = (stock_closing_prices[0])
    stocks_info[stock]["day 2 reading"] = (stock_closing_prices[1])
    stocks_info[stock]["day 3 reading"] = (stock_closing_prices[2])
    stocks_info[stock]["day 4 reading"] = (stock_closing_prices[3])
    stocks_info[stock]["day 5 reading"] = (stock_closing_prices[4])
    stocks_info[stock]["day 6 reading"] = (stock_closing_prices[5])
    stocks_info[stock]["current reading"] = (stock_closing_prices[6])
    
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["day 1 reading"])
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["day 2 reading"])
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["day 3 reading"])
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["day 4 reading"])
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["day 5 reading"])
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["day 6 reading"])
    stocks_info[stock]["list of readings"].append(stocks_info[stock]["current reading"])

    # if (
    #         "Stock has no ""historical data""!" not in stock_closing_info or 'Closing Price: NaN' not in stock_closing_info):
    #     plt.plot(list(stock_closing_info.keys()), list(stock_closing_info.values()), color=np.random.rand(3, ),
    #              label=stock)
    
    stock_count += 1

In [ ]:
# PrettyPrints the invalid stocks from the dictionary

# print(len(stocks_symbols))
invalid_stock_count = 0
for stock in stocks_symbols:
    if (len(stocks_info[stock]['reading dates']) != 7 or len(stocks_info[stock]['list of readings']) != 7):
        pprint(stocks_info[stock])
        print('------------------------')
        invalid_stock_count += 1
#         stocks_symbols.pop(stocks_symbols.index(stock))
        
# print(len(stocks_symbols))
print(invalid_stock_count)

In [ ]:
# Before deleting the invalid stocks from the dictionary and the list

print("Before:" + str(len(stocks_symbols)))
print("Before: " + str(len(stocks_info)))
# print('----------')

# stocks_symbols
# print('--------------------')
# stocks_info
# pprint(stocks_info)

duplicate_stocks_info = deepcopy(stocks_info)
duplicate_stocks_symbols = deepcopy(stocks_symbols)
# print(len(duplicate_stocks_info))

iter_count = 0

# Deletes the invalid stocks from the dictionary and the list

while (iter_count != invalid_stock_count):
    for stock in stocks_symbols:
        if (len(stocks_info[stock]['reading dates']) != 7 or len(stocks_info[stock]['list of readings']) != 7):
            stocks_symbols.remove(stock)
            invalid_stocks[stock] = stocks_info[stock]
            stocks_info.pop(stock)
            iter_count += 1

# After deleting the invalid stocks from the dictionary and the list
            
print("After: " + str(len(stocks_symbols)))
print("After: " + str(len(stocks_info)))


In [ ]:
# day1Readings.clear()
# day2Readings.clear()
# day3Readings.clear()
# day4Readings.clear()
# day5Readings.clear()
# day6Readings.clear()
# currentReadings.clear()

for stock in stocks_symbols:
    day1Readings.append(stocks_info[stock]["day 1 reading"])
    day2Readings.append(stocks_info[stock]["day 2 reading"])
    day3Readings.append(stocks_info[stock]["day 3 reading"])
    day4Readings.append(stocks_info[stock]["day 4 reading"])
    day5Readings.append(stocks_info[stock]["day 5 reading"])
    day6Readings.append(stocks_info[stock]["day 6 reading"])
    currentReadings.append(stocks_info[stock]["current reading"])

In [ ]:
# Creates a Pandas Dataframe

dataframe_dict = {"Stock": stocks_symbols, # stocks_symbols[:25]
                  f"CP Recording: {stock_closing_dates[0]}": day1Readings,
                  f"Day 2 CP Recording: {stock_closing_dates[1]}": day2Readings,
                  f"Day 3 CP Recording: {stock_closing_dates[2]}": day3Readings,
                  f"Day 4 CP Recording: {stock_closing_dates[3]}": day4Readings,
                  f"Day 5 CP Recording: {stock_closing_dates[4]}": day5Readings,
                  f"Day 6 CP Recording: {stock_closing_dates[5]}": day6Readings,
                  f"Today's CP Recording: {stock_closing_dates[6]}": currentReadings}


dataframe = pd.DataFrame(dataframe_dict)

# dataframe = dataframe.style.applymap(color_negative_red)

pd.set_option('display.max_rows', len(stocks_symbols))
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1)

print('---------------------------')
print('Dataframe has been created!')

In [ ]:
dataframe.index += 1
# dataframe_ascending = dataframe.sort_values(by=f"Today's CP Recording: {stock_closing_dates[6]}", ascending=True)
# dataframe_descending = dataframe.sort_values(by=f"Today's CP Recording: {stock_closing_dates[6]}", ascending=False)
dataframe.head(25)
# dataframe

In [ ]:
# for stock in stocks_symbols[:25]:
for stock in stocks_symbols:
    plt.plot(stocks_info[stock]["reading dates"], stocks_info[stock]["list of readings"], color = np.random.rand(3, ), label=stock)
    plt.title(f'Price of {stock} in USD from {stock_closing_dates[0]} to {stock_closing_dates[6]}')
    plt.xticks(rotation=45)
    plt.legend(loc='best')
#     plt.savefig(f'C:/Users/gorpa/Documents/Stocks/{stock}_stock.png', bbox_inches='tight')
    plt.show()

In [ ]:
# Saves/Exports the Pandas Dataframe and the Graphs

# Saves/Exports the Pandas Dataframe
print('\n')
dataframe.to_csv('C:/Users/gorpa/Desktop/Stocks/stocks_data.csv')
print("Finished exporting Pandas DataFrame to a CSV File")
dataframe.to_excel('C:/Users/gorpa/Desktop/Stocks/stocks_data.xlsx', sheet_name=f"From {stock_closing_dates[0]} to {stock_closing_dates[6]}")
print("Finished exporting Pandas DataFrame to an Excel Spreadsheet")

# Saves/Exports the Graphs/Plots

# for stock in stocks_symbols[:25]:
for stock in stocks_symbols:
    plt.plot(stocks_info[stock]["reading dates"], stocks_info[stock]["list of readings"], color = np.random.rand(3, ), label=stock)
    plt.title(f'Price of {stock} in USD from {stock_closing_dates[0]} to {stock_closing_dates[6]}')
    plt.xticks(rotation=45)
    plt.legend(loc='best')
    plt.savefig(f'C:/Users/gorpa/Desktop/Stocks/{stock}_stock.png', bbox_inches='tight')
    plt.close()
print("Finished exporting all matplotlib plots")